In [1]:
import pandas as pd
import numpy as np
#from sklearn.preprocessing import StandardScaler#, MinMaxScaler

In [2]:
dir_dados = '../2_dados/'
arq_sumario = dir_dados + 'data_sumario.parquet'
arq_serie = dir_dados + '/data_series/data_{}.npy'

In [3]:
arq_treino_sumario = dir_dados + '/treino_teste/treino_sumario.parquet'
arq_treino = dir_dados + '/treino_teste/treino.npy'
arq_teste_sumario = dir_dados + '/treino_teste/teste_sumario.parquet'
arq_teste = dir_dados + '/treino_teste/teste.npy'

### Abre arquivos

In [4]:
df = pd.read_parquet(arq_sumario)

In [5]:
df.head()

,signal_id,phase,target,arquivo
0,0,0,0,1
1,1,1,0,1
2,2,2,0,1
3,3,0,1,2
4,4,1,1,2


In [6]:
df['target'].value_counts()

0    466
1     35
Name: target, dtype: int64

In [7]:
df[['phase','target']].groupby(['phase','target']).size()

phase  target
0      0         156
       1          11
1      0         156
       1          11
2      0         154
       1          13
dtype: int64

### Preparação de dados

#### Divide em Treino e Teste

In [8]:
classe_neg_idx = df[df['target'] == 0]
classe_pos_idx = df[df['target'] == 1]

In [9]:
# Treino possui os primeiros exemplares da classe negativa
treino_prop = int(classe_neg_idx.shape[0] * 0.7)    # 70% da classe negativa
treino_idx = classe_neg_idx[:treino_prop]

In [10]:
# Teste possui os demais exemplares da classse negativa e todos os da positiva (descarga parcial)
teste_idx = classe_neg_idx[treino_prop:].append(classe_pos_idx)

In [11]:
def getSerie(df_index):
    dataset = []
    for _, linha in df_index.iterrows():
        serie = np.load(arq_serie.format(linha['arquivo']))
        fase = serie[linha['phase']]
        dataset.append(fase)
    return np.array(dataset)

In [12]:
treino = getSerie(treino_idx)
print('Dimensões de treino:', treino.shape)

Dimensões de treino: (326, 800000)


In [13]:
teste = getSerie(teste_idx)
print('Dimensões de teste:', teste.shape)

Dimensões de teste: (175, 800000)


#### Normaliza

In [14]:
#scaler = StandardScaler()   #StandardScaler(), MinMaxScaler()

In [15]:
#treino = np.swapaxes(treino, 0, 1)
#treino = scaler.fit_transform(treino)
#treino = np.swapaxes(treino, 0, 1)

In [16]:
#teste = np.swapaxes(teste, 0, 1)
#teste = scaler.fit_transform(teste)
#teste = np.swapaxes(teste, 0, 1)

In [17]:
treino = treino.astype(np.float32)
media = np.mean(treino, axis=1).reshape(-1, 1)
std = np.std(treino, axis=1).reshape(-1, 1)
treino = (treino - media) / std

In [18]:
teste = teste.astype(np.float32)
media = np.mean(teste, axis=1).reshape(-1, 1)
std = np.std(teste, axis=1).reshape(-1, 1)
teste = (teste - media) / std

#### Salva arquivos

In [19]:
treino_idx.to_parquet(arq_treino_sumario)
teste_idx.to_parquet(arq_teste_sumario)

In [20]:
np.save(arq_treino, treino)
np.save(arq_teste, teste)